In [1]:
!# Importing all the necessary Libraries
! pip install holidays
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import sys, math, os
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score    
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import svm
sys.path.append("..")
from sklearn.svm import SVC
from sklearn import metrics
import datetime as dt
%matplotlib inline
sns.set(style='ticks', palette='Set2')
from datetime import date 
import holidays 

In [7]:
df_2019= pd.read_csv("/Users/rajanpc/Desktop/DSBA_Project/201909-citibike-tripdata.csv")
df_2021= pd.read_csv("/Users/rajanpc/Desktop/DSBA_Project/202109-citibike-tripdata.csv")

station_name="E 41 St & Madison Ave""

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,327,2019-09-01 00:00:01.9580,2019-09-01 00:05:29.3410,3733,Avenue C & E 18 St,40.730563,-73.973984,504,1 Ave & E 16 St,40.732219,-73.981656,39213,Subscriber,1968,1
1,1145,2019-09-01 00:00:04.1430,2019-09-01 00:19:09.8360,3329,Degraw St & Smith St,40.682915,-73.993182,270,Adelphi St & Myrtle Ave,40.693083,-73.971789,21257,Customer,1969,0
2,1293,2019-09-01 00:00:07.3090,2019-09-01 00:21:40.7580,3168,Central Park West & W 85 St,40.784727,-73.969617,423,W 54 St & 9 Ave,40.765849,-73.986905,15242,Customer,1969,0
3,1753,2019-09-01 00:00:08.0640,2019-09-01 00:29:21.5040,3299,E 98 St & Park Ave,40.788130,-73.952060,3160,Central Park West & W 76 St,40.778968,-73.973747,38760,Subscriber,1990,1
4,613,2019-09-01 00:00:12.8510,2019-09-01 00:10:26.1850,486,Broadway & W 29 St,40.746201,-73.988557,478,11 Ave & W 41 St,40.760301,-73.998842,32094,Subscriber,1992,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,706,2019-09-01 00:54:07.4850,2019-09-01 01:05:54.2000,3400,E 110 St & Madison Ave,40.796154,-73.947821,3345,Madison Ave & E 99 St,40.789485,-73.952429,30017,Subscriber,1998,2
996,703,2019-09-01 00:54:15.6860,2019-09-01 01:05:59.5370,251,Mott St & Prince St,40.723180,-73.994800,432,E 7 St & Avenue A,40.726218,-73.983799,34949,Subscriber,1994,1
997,365,2019-09-01 00:54:18.6930,2019-09-01 01:00:24.6210,389,Broadway & Berry St,40.710446,-73.965251,3085,Roebling St & N 4 St,40.714690,-73.957390,32945,Subscriber,1985,2
998,420,2019-09-01 00:54:21.2600,2019-09-01 01:01:22.2350,236,St Marks Pl & 2 Ave,40.728419,-73.987140,2012,E 27 St & 1 Ave,40.739445,-73.976806,19206,Subscriber,1995,2


In [12]:
# Sorting the dataframe
df_2019.sort_values("tripduration", axis = 0, ascending = True, inplace = True, na_position ='last')

# Number of trips under 120 seconds
trip_dur = df_2019.apply(lambda x: True if x['tripduration'] < 121 else False , axis=1)
rows_num = len(trip_dur[trip_dur == True].index)
print('Rows having trip duration less than 120 seconds: ', rows_num)

df_2019_temp = df_2019.drop(df_2019[(df_2019['tripduration'] < 121)].index)
df_2019_cleaned = df_2019_temp[df_2019_temp['start station id'] == df_2019_temp['end station id']]
df_2019_cleaned.info()

Rows having trip duration less than 120 seconds:  36570
<class 'pandas.core.frame.DataFrame'>
Int64Index: 48688 entries, 1031253 to 2199923
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   tripduration             48688 non-null  int64  
 1   starttime                48688 non-null  object 
 2   stoptime                 48688 non-null  object 
 3   start station id         48688 non-null  int64  
 4   start station name       48688 non-null  object 
 5   start station latitude   48688 non-null  float64
 6   start station longitude  48688 non-null  float64
 7   end station id           48688 non-null  int64  
 8   end station name         48688 non-null  object 
 9   end station latitude     48688 non-null  float64
 10  end station longitude    48688 non-null  float64
 11  bikeid                   48688 non-null  int64  
 12  usertype                 48688 non-null  object 
 13  birth year  

In [13]:
#Create 'date' column
df_2019_cleaned['date'] = df_2019_cleaned['starttime'].apply(lambda x:x.split(' ')[0])

# Create 'hour' Column
df_2019_cleaned["hour"] = df_2019_cleaned['starttime'].apply(lambda x: x.split()[1].split(":")[0]) 

# Create 'day' column: the value of da is from 0 to 6: Mon - Sun
df_2019_cleaned['day'] = df_2019_cleaned['date'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d')).dt.weekday

# Change the format to datetime
df_2019_cleaned['date'] = pd.to_datetime(df_2019_cleaned['date'],format = '%Y-%m-%d')
df_2019_cleaned['starttime'] = pd.to_datetime(df_2019_cleaned['starttime'])
df_2019_cleaned['stoptime']= pd.to_datetime(df_2019_cleaned['stoptime'])

<ipython-input-13-58ef87367536>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019_cleaned['date'] = df_2019_cleaned['starttime'].apply(lambda x:x.split(' ')[0])
<ipython-input-13-58ef87367536>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019_cleaned["hour"] = df_2019_cleaned['starttime'].apply(lambda x: x.split()[1].split(":")[0])
<ipython-input-13-58ef87367536>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [17]:
week_end = []
for i in df_2019_cleaned['date']:
  num=i.weekday()
  if num < 5:
    k = 0
  else:
    k = 1   
  week_end.append(k)    
df_2019_cleaned['Weekend'] = week_end

holidays = holidays.UnitedStates(years = 2019) 
Holi_day = []
for i in df_2019_cleaned['date']:
  if i in us_holidays:
    Holi_day.append(1)
  else:
    Holi_day.append(0)  
df_2019_cleaned['Holiday'] = Holi_day 

Working_Day = []
for i in range(len(df_2019_cleaned['Weekend'])):
    if (df_2019_cleaned['Weekend'].iloc[i] == 1) or (df_2019_cleaned['Holiday'].iloc[i] == 1):
      Working_Day.append('FALSE')
    else:
      Working_Day.append('TRUE')
df_2019_cleaned['Working_Day'] = Working_Day




<ipython-input-17-5484c882fc38>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019_cleaned['Weekend'] = week_end
<ipython-input-17-5484c882fc38>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019_cleaned['Holiday'] = Holi_day
<ipython-input-17-5484c882fc38>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,bikeid,usertype,birth year,gender,date,hour,day,Weekend,Holiday,Working_Day
1031253,121,2019-09-14 16:08:07.505,2019-09-14 16:10:08.949,3092,Berry St & N 8 St,40.719009,-73.958525,3092,Berry St & N 8 St,40.719009,...,40578,Subscriber,1998,1,2019-09-14,16,5,1,0,FALSE
2297,121,2019-09-01 03:09:48.614,2019-09-01 03:11:50.343,3693,N 11 St & Kent Ave,40.722482,-73.959219,3693,N 11 St & Kent Ave,40.722482,...,15479,Customer,1990,1,2019-09-01,03,6,1,0,FALSE
2175539,121,2019-09-27 16:53:32.809,2019-09-27 16:55:34.795,3664,North Moore St & Greenwich St,40.720195,-74.010301,3664,North Moore St & Greenwich St,40.720195,...,17623,Subscriber,1982,2,2019-09-27,16,4,0,0,TRUE
1991909,121,2019-09-25 16:34:49.609,2019-09-25 16:36:50.975,426,West St & Chambers St,40.717548,-74.013221,426,West St & Chambers St,40.717548,...,31515,Customer,1982,2,2019-09-25,16,2,0,0,TRUE
1766483,121,2019-09-23 07:28:39.210,2019-09-23 07:30:40.747,394,E 9 St & Avenue C,40.725213,-73.977688,394,E 9 St & Avenue C,40.725213,...,26970,Subscriber,1993,1,2019-09-23,07,0,0,0,TRUE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39113,169364,2019-09-01 16:39:48.231,2019-09-03 15:42:33.159,3379,E 103 St & Lexington Ave,40.790305,-73.947558,3379,E 103 St & Lexington Ave,40.790305,...,39565,Customer,1969,0,2019-09-01,16,6,1,0,FALSE
2212829,217128,2019-09-28 01:38:24.055,2019-09-30 13:57:12.761,3347,Van Brunt St & Wolcott St,40.677343,-74.012751,3347,Van Brunt St & Wolcott St,40.677343,...,15512,Customer,1969,0,2019-09-28,01,5,1,0,FALSE
1940756,259565,2019-09-25 05:39:37.042,2019-09-28 05:45:42.965,3637,Fulton St & Waverly Ave,40.683239,-73.965996,3637,Fulton St & Waverly Ave,40.683239,...,30344,Customer,1969,0,2019-09-25,05,2,0,0,TRUE
183519,347812,2019-09-03 23:18:06.998,2019-09-07 23:54:59.284,436,Hancock St & Bedford Ave,40.682166,-73.953990,436,Hancock St & Bedford Ave,40.682166,...,32874,Customer,1969,0,2019-09-03,23,1,0,0,TRUE


In [36]:
#Weather

dfw= pd.read_csv("/Users/rajanpc/Desktop/DSBA_Project/weather_nyc_20180101_to_20211031.csv")

dfw = dfw[(dfw['DATE']> "2018-12-31") & (dfw['DATE']< "2020-01-01")]

dfw['DATE']= pd.to_datetime(dfw['DATE']) 

#Add calculated column with the average temp = (TMAX + TMIN)/2
dfw['COLD'] = (dfw['TMAX'] + dfw['TMIN'])/2

# Leave only important columns out of 'NAME','DATE', 'TEMP', 'PRCP', 'SNOW', 'PREC', 'COLD'
dfw = dfw.loc[:, dfw.columns.intersection(['NAME','DATE', 'PRCP', 'COLD'])]


In [37]:
df_2019_final = pd.merge(df_2019_cleaned[['date','start station name','end station name','Weekend','Holiday','Working_Day']], dfw[['DATE','COLD','PRCP']],how='left',left_on='date', right_on='DATE')
df_2019_final = df_2019_final.drop(columns='DATE')

,date,start station name,end station name,Weekend,Holiday,Working_Day,COLD,PRCP
0,2019-09-14,Berry St & N 8 St,Berry St & N 8 St,1,0,FALSE,67.5,0.00
1,2019-09-01,N 11 St & Kent Ave,N 11 St & Kent Ave,1,0,FALSE,70.5,0.00
2,2019-09-27,North Moore St & Greenwich St,North Moore St & Greenwich St,0,0,TRUE,67.5,0.00
3,2019-09-25,West St & Chambers St,West St & Chambers St,0,0,TRUE,68.5,0.00
4,2019-09-23,E 9 St & Avenue C,E 9 St & Avenue C,0,0,TRUE,79.5,0.11
...,...,...,...,...,...,...,...,...
48683,2019-09-01,E 103 St & Lexington Ave,E 103 St & Lexington Ave,1,0,FALSE,70.5,0.00
48684,2019-09-28,Van Brunt St & Wolcott St,Van Brunt St & Wolcott St,1,0,FALSE,73.5,0.00
48685,2019-09-25,Fulton St & Waverly Ave,Fulton St & Waverly Ave,0,0,TRUE,68.5,0.00
48686,2019-09-03,Hancock St & Bedford Ave,Hancock St & Bedford Ave,0,0,TRUE,74.5,0.00


In [ ]:
Lexington Ave & E 44th St, Madison Ave & E 41st St, 5th Ave & W 44th St, 
Park Ave & E 40th St, 
Park Ave & E 47th St